# Introduction

O

# Configuration information

# launch.json: https://code.visualstudio.com/docs/python/debugging

# Libraries

In [1]:
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pvlib
import pvlib.pvsystem as pvsystem
from scipy.optimize import brentq, newton

# Global Variables

## Setup Variables

In [2]:
CREST_IRRADIANCE_FOLDER = r"C:\Users\wsfm\OneDrive - Loughborough University\_Personal_Backup\python_repositories\ground-based-solar-irradiance\assets\isc_irradiance_files"
CREST_IRRADIANCE_FILE = r"chpoa_y15.csv"
CREST_IRR_FILEPATH = os.path.join(CREST_IRRADIANCE_FOLDER, CREST_IRRADIANCE_FILE)

# PVsyst project exported meteo file
PVSYST_HOURLY_METEO = r"Loughborough_meteo.csv"
PS_WEATHER_FILEPATH = os.path.join(os.getcwd().replace("examples","data"),PVSYST_HOURLY_METEO)

PS_HOURLY_FOLDERPATH = r"C:\Users\wsfm\PVsyst7.0_Data\UserHourly"
# with old sport data
PVSYST_HOURLY_FILE = r"Project_test_221012_Project_VC0_HourlyRes_spot.csv"
# with replaced average
PVSYST_HOURLY_FILE = r"Project_test_221012_Project_VC0_HourlyRes_250_m10_dummy.csv"
# 26/3/23 power output in W
PVSYST_HOURLY_FILE = r"Project_test_221012_Project_VC4_HourlyRes_250_1m.CSV"

PS_HOURLY_FILEPATHS = [os.path.join(PS_HOURLY_FOLDERPATH,PVSYST_HOURLY_FILE)]


MODULES_FOLDER = r"C:\Users\wsfm\PVsyst7.0_Data\ComposPV\PVmodules"


#pvlib dictionary, function parameters to columns
# 27/7/23 to be checked what used, AOI useful for uncertainty later
PLD:dict = {
"aoi":"AngInc", #if calculated from pvsyst
"apparent_zenith":"apparent_zenith",
"airmass_absolute":"airmass_absolute",
"dhi":"DiffHor", 
"dni":"BeamHor", 
"dni_extra":"dni_extra", 
"effective_irradiance":"g_cmp11_ppuk", #TBC
"ghi":"GlobHor", 
"poa_global": "g_cmp11_ppuk",
"poa_direct": "poa_direct",
"poa_diffuse": "poa_diffuse",
"solar_azimuth":"AzSol", #solar_azimuth
"solar_zenith":"solar_zenith",
"sky_diffuse":"DifITrp",
"temp_air":"T_Amb", 
"temp_cell":"temp_cell", #TArray in PVsyst 
"time":"datetime",
"wind_speed":"WindVel"
}

#17/7/23 print only the ones not having keys = values

# methods include also model parameters
METHODS:dict = {
    "extra_radiation": "spencer", #'pyephem', 'spencer', 'asce', 'nrel'
    "iv": "desoto", #pvsyst, #cecmod
    "temp_cell": "sapm",
    "total_irradiance": "haydavies" #model
}


# verified
MODULE_NAMES = ["SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_mono",
           "SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_poly"]
MODULE_NAME = MODULE_NAMES[1]
SAPM_MODULE_TEMPERATURE = 'open_rack_glass_polymer'


## Constants

In [3]:
DEGDT = - 0.0002677 #The temperature dependence of the energy bandgap at reference conditions in units of 1/K
EGREF = 1.12 #The energy bandgap at reference temperature in units of eV. 1.121 eV for crystalline silicon. EgRef must be >0.
IRRAD_REF = 1000
IVCURVE_PNTS = 0 #400
K_BOLTZMANN = 1.381 / (10 ** 23) # J/K
Q_ELECTRON = 1.602 / (10 ** 19) # C
SOLAR_CONSTANT = 1366.1
TEMP_REF = 25 #C°

## System Setup

In [4]:
# PVSYST Loughborough
LATITUDE = 52.7767 #29/7/23 before was negative
LONGITUDE = -1.2
ALTITUDE = 53

# PVSYST convention
SURFACE_TILT = 34
SURFACE_AZIMUTH = 0

# Module data

In [5]:
# Load the database of CEC module model parameters
modules = pvsystem.retrieve_sam('cecmod')
# the dict keys are module names, and the values are model parameters for that module
module = modules[MODULE_NAME]
# load SAM temperature model parameters into module
module["sapm"] = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm'][SAPM_MODULE_TEMPERATURE]

In [6]:
module

Technology                                Multi-c-Si
Bifacial                                           0
STC                                          245.168
PTC                                            220.1
A_c                                              1.6
Length                                         1.645
Width                                           0.97
N_s                                               60
I_sc_ref                                        8.49
V_oc_ref                                        37.5
I_mp_ref                                        7.96
V_mp_ref                                        30.8
alpha_sc                                    0.007047
beta_oc                                    -0.145875
T_NOCT                                          46.6
a_ref                                       1.643428
I_L_ref                                      8.49537
I_o_ref                                          0.0
R_s                                         0.

In [7]:
module["latitude"] = LATITUDE
module["longitude"] = LONGITUDE
module["altitude"] = ALTITUDE
module["surface_tilt"] = SURFACE_TILT
module["surface_tilt"] = SURFACE_AZIMUTH
# surface tilt to be added



In [8]:
module

Technology                                  Multi-c-Si
Bifacial                                             0
STC                                            245.168
PTC                                              220.1
A_c                                                1.6
Length                                           1.645
Width                                             0.97
N_s                                                 60
I_sc_ref                                          8.49
V_oc_ref                                          37.5
I_mp_ref                                          7.96
V_mp_ref                                          30.8
alpha_sc                                      0.007047
beta_oc                                      -0.145875
T_NOCT                                            46.6
a_ref                                         1.643428
I_L_ref                                        8.49537
I_o_ref                                            0.0
R_s       

# Wrap-up indipendent functions

In [9]:
def pvl_azm_from_pvs(pvs_azm):
    """
    Azimutal conversion for pvlib when using pvsyst convention
    """ 
    #24/1/23 updated
    return (pvs_azm+540)%360

# 25/7/23 not used
def add_effective_irradiance(df:pd.DataFrame, module:pd.Series, pld:dict=PLD) -> pd.DataFrame:
    """
    wrap-up function to calculate the SAPM effective irradiance using the SAPM spectral loss and SAPM angle of incidence loss functions.
    :param df:
    :pld: pvlib parameters -> columns
    :module: pvlib 
    """
    df["sapm_effective_irradiance"] = df.apply(lambda x: pvlib.pvsystem.sapm_effective_irradiance(               
                        poa_direct = x[pld["poa_direct"]], # x['poa_direct'], # x["BeamTrp"]
                        poa_diffuse =  x[pld["poa_diffuse"]],  #x['poa_diffuse'], # x["DifITrp"] + x["DifSInc"] 
                        airmass_absolute = x[pld["airmass_absolute"]], #x["am_abs"],
                        aoi = x[pld["aoi"]],   #x["AngInc"], #x["aoi"],
                        module = module), axis=1)
    return df

# 25/7/23 not used
def add_extra_radiation(df:pd.DataFrame, methods:pd.Series=METHODS, pld:dict=PLD) -> pd.DataFrame:
      df["dni_extra"] = df.apply(lambda x: pvlib.irradiance.get_extra_radiation(
            df["date"], solar_constant=SOLAR_CONSTANT, method=methods["extra_radiation"], epoch_year=x["year"]), axis=1)
      return df

def add_temp_cell(df:pd.DataFrame, module:pd.Series, methods:pd.Series=METHODS, pld:dict=PLD) -> pd.DataFrame:
    if methods["temp_cell"] == "sapm":
        temperature_model_parameters = module["sapm"]
        df["temp_cell"] = df.apply(lambda x: pvlib.temperature.sapm_cell(
                        poa_global=x[pld["poa_global"]], 
                        temp_air=x[pld["temp_air"]], 
                        wind_speed=x[pld["wind_speed"]], 
                        **temperature_model_parameters), axis=1)
    return df


# to be updated with pld
def add_iv(df:pd.DataFrame, module:pd.Series, methods:pd.Series=METHODS, pld:dict=PLD,
           ivcurve_pnts=IVCURVE_PNTS, k_Boltzmann=K_BOLTZMANN, 
           temp_ref=TEMP_REF, irrad_ref=IRRAD_REF, q_electron=Q_ELECTRON, dEgdT=DEGDT,
           EgRef=EGREF
           ) -> pd.DataFrame:
    """
    wrap-up function
    :param df:
    :param x: dataframe dictionary
    :param ps_prms: parameters dictionary 
    :param calcparams: "desoto" or "pvsyst"
    :param calcmp: "singlediode", "bishop88"
    :param method: 'lambertw' #, 'newton', or 'brentq'
    :param ivcurve_pnts: 
    :param cecmodule:
    :param k_Boltzmann:
    :param t_ref: in C° transformed later
    :param q_electron:
    :return: add columns to df (see below). i & v as array for each point.
    """ 

    # using the ones from pvsyst instead of pvlib
    # no need to order
    clms_sd = ["photocurrent","saturation_current","resistance_series","resistance_shunt","nNsVth"]
    clms = ["i_sc","v_oc","i_mp","v_mp","p_mp","i_x","i_xx"]
    if ivcurve_pnts > 0: clms = clms + ["v","i"]

    calcparams = methods["iv"]
    iv_keys = ["alpha_sc","I_L_ref", "I_o_ref", "R_sh_ref", "R_s"]
    iv_parameters = {key: module[key] for key in iv_keys}
    iv_parameters["a_ref"] = module["gamma_r"] * module["N_s"] * k_Boltzmann * (temp_ref+ 273.15) / q_electron,
    iv_parameters["dEgdT"] = dEgdT
    iv_parameters["irrad_ref"] = irrad_ref
    iv_parameters["temp_ref"] = temp_ref
    iv_parameters["EgRef"] = EgRef

    if calcparams == "desoto2":
        df[clms_sd] = df.apply(
        lambda x:                 
        pvlib.pvsystem.calcparams_desoto(
        effective_irradiance = x[pld["effective_irradiance"]], 
        temp_cell = x[pld["temp_cell"]], 
        **iv_parameters
        ) if x[pld["effective_irradiance"]] != 0 else
        pd.Series(dict(zip(clms_sd,[np.nan]*len(clms_sd)))), 
        axis=1, result_type='expand')
        

    elif calcparams == "desoto":
        df[clms_sd] = df.apply(
        lambda x:                 
        pvlib.pvsystem.calcparams_desoto(
        effective_irradiance = x[pld["effective_irradiance"]], 
        temp_cell = x[pld["temp_cell"]], 
        alpha_sc = module["alpha_sc"], #The short-circuit current temperature coefficient of the module in units of A/C.
        a_ref =  module["gamma_r"] * module["N_s"] * k_Boltzmann * (temp_ref+ 273.15) / q_electron,
        # The product of the usual diode ideality factor (n, unitless), number of cells in series (Ns), and cell thermal voltage at reference conditions, in units of V        
        # {\displaystyle V_{\text{T}}=kT/q,} the thermal voltage.
        # gamma_ref = module["gamma_ref"], #The diode ideality factor
        # mu_gamma = module["mu_gamma"], #The temperature coefficient for the diode ideality factor, 1/K
        I_L_ref = module["I_L_ref"], #The light-generated current (or photocurrent) at reference conditions, in amperes.
        I_o_ref = module["I_o_ref"], #The dark or diode reverse saturation current at reference conditions, in amperes.
        R_sh_ref = module["R_sh_ref"], #The shunt resistance at reference conditions, in ohms.
        # R_sh_0 = module["R_sh_0"], #The shunt resistance at zero irradiance conditions, in ohms.
        R_s = module["R_s"], #The series resistance at reference conditions, in ohms.
        # cells_in_series = module["cells_in_series"], #The number of cells connected in series.
        #R_sh_exp=module["R_sh_exp"], #The exponent in the equation for shunt resistance, unitless. Defaults to 5.5.
        dEgdT= dEgdT, #The temperature dependence of the energy bandgap at reference conditions in units of 1/K
        EgRef=EgRef, #The energy bandgap at reference temperature in units of eV. 1.121 eV for crystalline silicon. EgRef must be >0.
        irrad_ref=irrad_ref, # Reference irradiance in W/m^2.
        temp_ref=temp_ref # Reference cell temperature in C.
        ) if x[pld["effective_irradiance"]] != 0 else
        pd.Series(dict(zip(clms_sd,[np.nan]*len(clms_sd)))), 
        axis=1, result_type='expand')

    # version 0.93, PVsyst v6 model
    elif calcparams == "pvsyst":
        df[clms_sd] = df.apply(lambda x:                 
        pvlib.pvsystem.calcparams_pvsyst(
        effective_irradiance = x[pld["effective_irradiance"]], 
        temp_cell = x[pld["temp_cell"]], 
        alpha_sc = module["alpha_sc"], #The short-circuit current temperature coefficient of the module in units of A/C.
        gamma_ref = module["gamma_r"], #The diode ideality factor
        mu_gamma = module["mu_gamma"], #The temperature coefficient for the diode ideality factor, 1/K
        I_L_ref = module["I_L_ref"], #The light-generated current (or photocurrent) at reference conditions, in amperes.
        I_o_ref = module["I_o_ref"], #The dark or diode reverse saturation current at reference conditions, in amperes.
        R_sh_ref = module["R_sh_ref"], #The shunt resistance at reference conditions, in ohms.
        R_sh_0 = module["R_sh_0"], #The shunt resistance at zero irradiance conditions, in ohms.
        R_s = module["R_s"], #The series resistance at reference conditions, in ohms.
        cells_in_series = module["cells_in_series"], #The number of cells connected in series.
        R_sh_exp=module["R_sh_exp"], #The exponent in the equation for shunt resistance, unitless. Defaults to 5.5.
        EgRef=module["EgRef"], #The energy bandgap at reference temperature in units of eV. 1.121 eV for crystalline silicon. EgRef must be >0.
        irrad_ref=module["irrad_ref"], # Reference irradiance in W/m^2.
        temp_ref=module["temp_ref"] # Reference cell temperature in C.
        ) if x[pld["effective_irradiance"]] != 0 else
        pd.Series(dict(zip(clms_sd,[np.nan]*len(clms_sd)))), 
        axis=1, result_type='expand')

    elif calcparams == "cecmod":
        modules = pvlib.pvsystem.retrieve_sam('CECmod')       
        #swmod = modules[cecmodule]
        df[clms_sd] = df.apply(lambda x:                 
        pvlib.pvsystem.calcparams_cec(
        effective_irradiance = x[pld["effective_irradiance"]], 
        temp_cell = x[pld["temp_cell"]],       
        alpha_sc = module["alpha_sc"], #The short-circuit current temperature coefficient of the module in units of A/C.
        a_ref = module["gamma_r"] * module["cells_in_series"] * k_Boltzmann * t_ref / q_electron,
        I_L_ref = module["I_L_ref"], #The light-generated current (or photocurrent) at reference conditions, in amperes.
        I_o_ref = module["I_o_ref"], #The dark or diode reverse saturation current at reference conditions, in amperes.
        R_sh_ref = module["R_sh_ref"], #The shunt resistance at reference conditions, in ohms.
        R_s = module["R_s"], #The series resistance at reference conditions, in ohms.
        Adjust = module["Adjust"]
        ) if x[pld["effective_irradiance"]] != 0 else
        pd.Series(dict(zip(clms_sd,[np.nan]*len(clms_sd)))), 
        axis=1, result_type='expand')

    df[clms] = df.apply(lambda x:                 
    pvlib.pvsystem.singlediode(
    photocurrent = x["photocurrent"],
    saturation_current = x["saturation_current"],
    resistance_series = x["resistance_series"],
    resistance_shunt = x["resistance_shunt"],
    nNsVth = x["nNsVth"], # k_Boltzmann * x[pld["temp_cell] / q_electron
    ivcurve_pnts = ivcurve_pnts,
    method = "lambertw"
    ) if x[pld["effective_irradiance"]] != 0 else
    pd.Series(dict(zip(clms,[np.nan]*len(clms))))
    , axis=1, result_type='expand')
    # not clear why array returned
    df["i_mp"]=df["i_mp"].apply(float)
    df["v_mp"]=df["v_mp"].apply(float)

    return df



# 25/7/23 not used
def add_solar_position(df:pd.DataFrame, module:pd.Series, pld:dict=PLD) -> pd.DataFrame:
    """
    wrap-up function to get solar position
    """
    exss = []
    crts = []

    clms = ["apparent_zenith","zenith","apparent_elevation","elevation","azimuth","equation_of_time"]

    if any([clms not in df.columns.to_list()]):    
            for c in clms: exss.append(c) if c in df.columns.to_list() else crts.append(c)

            df[clms] = df.apply(
            lambda x: pvlib.solarposition.get_solarposition(
                            time= x[pld["datetime"]],
                            latitude = module["latitude"],
                            longitude = module["longitude"],
                            altitude = module["altitude"],
                            temperature= x[pld["temp_air"]],
                            pressure=pvlib.atmosphere.alt2pres(module["altitude"])).values[0], axis=1, result_type='expand')

    elif all([clms not in df.columns.to_list()]): 
            for c in clms: 
                    exss.append(c)

    # solar_zenith
    c = "airmass"
    if c not in df.columns.to_list(): 
            df[c] = df['apparent_zenith'].apply(lambda x: pvlib.atmosphere.get_relative_airmass(x))
            crts.append(c)

    elif c in df.columns.to_list(): exss.append(c) 

    c = "pressure"
    if c not in df.columns.to_list(): 
            df[c] = pvlib.atmosphere.alt2pres(module["altitude"])
            crts.append(c)

    elif c in df.columns.to_list(): exss.append(c) 

        
    c = "airmass_absolute"
    if c not in df.columns.to_list(): 
            # system['surface_tilt'], system['surface_azimuth'],  solpos["apparent_zenith"], solpos["azimuth"],
            df[c] = df.apply(lambda x: pvlib.atmosphere.get_absolute_airmass(x["airmass"], x["pressure"]), axis=1)
            crts.append(c)
    
    elif c in df.columns.to_list(): exss.append(c) 
    
    c = "aoi"
    # aoi already existing due to previously: 
    # df["aoi"] = df.apply(lambda x: pvlib.irradiance.aoi(SURFACE_TILT, pvl_azm_from_pvs(SURFACE_AZIMUTH),x["solar_zenith"], x["solar_azimuth"]), axis=1)
    # 30/1/23 why apparent zenith to be used ?
    if c not in df.columns.to_list(): 
            # 7/1/23 latitude & longitude pvlib vs pvsyst to be adpated ?
            df[c] = df.apply(lambda x: pvlib.irradiance.aoi(
                    SURFACE_TILT, 
                    pvl_azm_from_pvs(SURFACE_AZIMUTH),
                    x["zenith"],    #x["apparent_zenith"],
                    x["azimuth"]), axis=1)
            crts.append(c)
    
    elif c in df.columns.to_list(): exss.append(c) 

    print("Existing: "+", ".join(exss)+"\n"+"Created: "+", ".join(crts))

    return df


# 25/7/23 not used
def add_total_irradiance(df:pd.DataFrame, module:pd.Series, methods:dict=METHODS, pld:dict=PLD) -> pd.DataFrame:
    """
    wrap-up function to get total irradiance
    """
    exss = []
    crts = []

    clms = ['poa_global', 'poa_direct', 'poa_diffuse', 'poa_sky_diffuse', 'poa_ground_diffuse']

    if any([clms not in df.columns.to_list()]):    
            for c in clms: exss.append(c) if c in df.columns.to_list() else crts.append(c)
            # 15/1/23 total irradiance is dictionary/series
            # 7/1/23 total irradiance expected smaller than POA measured due to Hay inverted
            df[clms] = df.apply(
                    lambda x: pvlib.irradiance.get_total_irradiance(
                    module["surface_tilt"], 
                    pvl_azm_from_pvs(module["surface_azimuth"]),
                    x[pld["solar_zenith"]], #x["apparent_zenith"], 
                    x[pld["solar_azimuth"]],
                    x[pld["dni"]], 
                    x[pld["ghi"]],
                    x[pld["dhi"]],
                    dni_extra=x[pld["dni_extra"]] * pvlib.tools.cosd(x[pld["apparent_zenith"]]),
                    model=methods["total_irradiance"]
                    ), axis=1, result_type='expand')
    
    elif all([clms not in df.columns.to_list()]): 
            for c in clms: 
                    exss.append(c)

    print("Existing: "+", ".join(exss)+"\n"+"Created: "+", ".join(crts))

    return df



def get_ps_prms(folderpath, filename) -> dict:
    """
    wrap-up function for pvsystem moduel parameters
    :
    :
    :return: parameters
    
    Parameters not included in PAN files to be added manually:

    #The dark or diode reverse saturation current at reference conditions, in amperes.
    "I_o_ref" : 0.040 / (10 ** 9)

    #The energy bandgap at reference temperature in units of eV. 1.121 eV for crystalline silicon. EgRef must be >0.
    "EgRef" : 1.12, #1

    # Reference irradiance in W/m^2.
    "temp_ref" : 25
    """
    ps_prms = {}
    f = open(os.path.join(folderpath,filename))
    for r in f.readlines():
        s = r.split("=")
        if len(s) > 1:
            ps_prms[s[0].strip()] = s[1].rstrip()
    f.close()
    return ps_prms


def get_weather_df(crest_irr_filepath:str=CREST_IRR_FILEPATH, 
                   ps_weather_filepath:str=PS_WEATHER_FILEPATH, ps_hourly_filepaths:list=PS_HOURLY_FILEPATHS) -> pd.DataFrame:
    """
    wrap-up to create a weather file
    :param crest_filepath: CREST_IRRADIANCE_FOLDER,CREST_IRRADIANCE_FILE
    :param ps_weather_filepath: os.path.join(os.getcwd().replace("examples","data"),PVSYST_HOURLY_METEO)
    :param ps_hourly_filepath: PVSYST_FOLDER
    """

    #Data for pvsyst imported from:
    #C:\Users\wsfm\OneDrive - Loughborough University\_Personal_Backup\model_predict\ground-based-solar-irradiance\assets\isc_irradiance_files\*.*
    crest_irr = pd.read_csv(filepath_or_buffer=crest_irr_filepath, 
    encoding="utf-8-sig",delimiter=",", encoding_errors="replace") #, usecols=[0,1,2,3,4,5,6]) #nrows=20,

    # 22/12/22 UTC false initially to compare with PVsyst
    # using date since also in pvsyst
    crest_irr["date"] = crest_irr["date_trunc"].apply(lambda x: pd.to_datetime(x, format=r"%d/%m/%Y %H:%M:%S", utc=False))

    crest_irr["year"] = crest_irr["date"].dt.year

    # reading meteo data
    pvs_mt = pd.read_csv(filepath_or_buffer=ps_weather_filepath, 
    encoding="utf-8-sig",delimiter=";", encoding_errors="replace", usecols=[0,1,2,3,4,5,6]) #nrows=20,
    pvs_mt["Interval beginning"] = pvs_mt["Interval beginning"].apply(lambda x: pd.to_datetime(x, format=r"%d/%m/%y %H:%M", utc=False))
    pvs_mt.rename(columns={"Interval beginning":"date"}, inplace=True)
    # Direct normal irradiance (DNI or BeamNor)  are also available for concentrating systems.

    # reading output data
    # C:\Users\wsfm\PVsyst7.0_Data\Models

    pvss = {}
    pvs_slcs = {}
    # 20/3/23 for standard analysis only first file will be considered

    for f in ps_hourly_filepaths:
        # f = PVSYST_HOURLY_FILE
        pvs_out = pd.read_csv(filepath_or_buffer=f, encoding="utf-8-sig",
        skiprows=10,   delimiter=",", encoding_errors="replace") #nrows=20,
        pvs_out = pvs_out.iloc[1:]
        for c in pvs_out.columns.to_list():
            if c != "date": 
                pvs_out[c] = pvs_out[c].astype(float)
        # PVsyst use local time
        # https://forum.pvsyst.com/topic/175-time-zone-in-met-data-file-and-site-file/
        pvs_out["date"] = pvs_out["date"].apply(lambda x: pd.to_datetime(x, format=r"%d/%m/%y %H:%M", utc=False))

        #MERGING DATAFRAMES

        # merging to get solar elevation for extracting I0
        # using out as base since mt has only up to decimal
        pvs_mt_mrg_columns = ["date"]+[c for c in pvs_mt.columns.to_list() if c not in pvs_out.columns.to_list()]

        # different merging to check df well aligned
        # pvs_out_mrg_columns2 = [c for c in pvs_out.columns.to_list()]

        pvs = pvs_out.merge(pvs_mt.loc[:,pvs_mt_mrg_columns], left_on="date", right_on="date", how="left", suffixes=('_x', '_y'))

        # merging for comparison
        pvs = pvs.merge(crest_irr[["g_cmp11_ppuk","date"]], left_on="date", right_on="date")

        pvs.rename(columns={"date":"datetime"}, inplace=True)
        pvs["year"] = pvs["datetime"].dt.year
        pvs["date"] = pvs["datetime"].dt.date
        pvs["hour"] = pvs["datetime"].dt.hour
        # test day previously selected
        pvss[f] = pvs
        
        # test selection
        #pvs_slc = pvs.loc[(pvs["datetime"].dt.day==22) & (pvs["datetime"].dt.month==3),:].copy(deep=True)
        #pvs_slcs[f] = pvs_slc
        return pvs

# Loading & preparing weather data

In [10]:
pvs = get_weather_df()

## Sampling weather data

In [11]:
pvs_slc = pvs.loc[(pvs["datetime"].dt.day==22) & (pvs["datetime"].dt.month==3),:].copy(deep=True)


# Effective irradiance modeling

Only total POA is monitored, used to model cell temperature.

In [12]:
"""if "poa_diffuse" not in pvs.columns.to_list():
    pvs["poa_diffuse"] = pvs["DifITrp"] + pvs["CircTrp"]
pvs_slc = add_effective_irradiance(pvs_slc, module)"""

'if "poa_diffuse" not in pvs.columns.to_list():\n    pvs["poa_diffuse"] = pvs["DifITrp"] + pvs["CircTrp"]\npvs_slc = add_effective_irradiance(pvs_slc, module)'

# Temperature modeling

In [13]:
pvs_slc = add_temp_cell(pvs_slc, module)

# IV-curves modeling

In [14]:
pvs_slc = add_iv(pvs_slc, module)

c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-packages\pvlib\tools.py:358: RuntimeWarning: All-NaN axis encountered
  iterlimit = 1 + np.nanmax(
c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-packages\pvlib\tools.py:358: RuntimeWarning: All-NaN axis encountered
  iterlimit = 1 + np.nanmax(
c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-packages\pvlib\tools.py:358: RuntimeWarning: All-NaN axis encountered
  iterlimit = 1 + np.nanmax(
c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-packages\pvlib\tools.py:358: RuntimeWarning: All-NaN axis encountered
  iterlimit = 1 + np.nanmax(
c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-packages\pvlib\tools.py:358: RuntimeWarning: All-NaN axis encountered
  iterlimit = 1 + np.nanmax(
c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-packages\pvlib\tools.py:358: RuntimeWarning: All-NaN axis encountered
  iterlimit = 1 + np.nanmax(
c:\ProgramData\Anaconda3\envs\pv_module_modeling\lib\site-

In [15]:
"""
example retrieving data from sam
modules = pvlib.pvsystem.retrieve_sam('CECmod')
ps_prms_cec = modules['SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_250_poly']
"""

"\nexample retrieving data from sam\nmodules = pvlib.pvsystem.retrieve_sam('CECmod')\nps_prms_cec = modules['SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_250_poly']\n"

# Annex - Modules description in CEC

## Modules characteristics


|Model|Technology|Isc(A)|Voc(V)|Pmax(W)|Channel|records(most)|start|
|---|---|---|---|---|---|---|---|
|Solarworld_SW245_mono|c-Si|8.25|37.7|245|1,2|2824341|15/04/2013  11:33:00|
|Solarworld_SW245_poly|mc-Si|8.49|37.5|245|3,4|2766361|15/04/2013 12:24|
|Trina_TSM-235PC05|mc-Si|8.55|37.2|235|5,6|2771870|15/04/2013  12:43:02|
|Trina_TSM-180DC01A|c-Si|5.35|44.2|180|7,8|2640722|25/04/2013  15:15:30|
|Trina_TSM-195DC80.08|c-Si|5.70|45.4|195|9,10|2808023|11/03/2013  11:57:15|
|Kyocera_KD240GH-2PB|mc-Si|8.59|36.9|240|13,14|2393661|20/05/2013  16:17:00|


Source file: PPUK_Module_List_2013-12-13-BG.xlsx


In [16]:
# see PPUK_Module_List_2013-12-13-BG for details why poly chosen
mkws = [
["SolarWorld","245"],
["Trina","235", "05"],
["Trina","180"],
["Trina","195"],
["Kyocera", "240"]
]

for kws in mkws:
    ms = [m for m in modules.columns.to_list() if all(ks in m for ks in kws)]
    print(modules.loc[["I_sc_ref","V_oc_ref","I_mp_ref","V_mp_ref"],ms])

         SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_mono  \
I_sc_ref                                               8.25      
V_oc_ref                                               37.7      
I_mp_ref                                               7.96      
V_mp_ref                                               30.8      

         SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_mono_black  \
I_sc_ref                                               8.25            
V_oc_ref                                               37.7            
I_mp_ref                                               7.96            
V_mp_ref                                               30.8            

         SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_poly  
I_sc_ref                                               8.49     
V_oc_ref                                               37.5     
I_mp_ref                                               7.96     
V_mp_ref                                        

# Module description to use Sandia model for effective irradiance

In [17]:
# Load the database of CEC module model parameters
modules = pvsystem.retrieve_sam('cecmod')

mkws = [
["SolarWorld","245"],
["Trina","235", "05"],
["Trina","180"],
["Trina","195"],
["Kyocera", "240"]
]

for kws in mkws:
    ms = [m for m in modules.columns.to_list() if all(ks in m for ks in kws)]
    print(modules.loc[["I_sc_ref","V_oc_ref","I_mp_ref","V_mp_ref"],ms])

         SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_mono  \
I_sc_ref                                               8.25      
V_oc_ref                                               37.7      
I_mp_ref                                               7.96      
V_mp_ref                                               30.8      

         SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_mono_black  \
I_sc_ref                                               8.25            
V_oc_ref                                               37.7            
I_mp_ref                                               7.96            
V_mp_ref                                               30.8            

         SolarWorld_Industries_GmbH_Sunmodule_Plus_SW_245_poly  
I_sc_ref                                               8.49     
V_oc_ref                                               37.5     
I_mp_ref                                               7.96     
V_mp_ref                                        

# Loading module parameters

In [ ]:
# examples provided for SW 245 Poly
# Beware that showed value in pvsyst is approximation like: muGamma, RSerie, muISC

pvs_to_pvl = {
#The short-circuit current temperature coefficient of the module in units of A/C.
# muIsc  (often named Alpha)
"alpha_sc" : "muISC", #2.9 / (10**3), 
#The diode ideality factor # muIsc  (often named Alpha)
"gamma_ref" : 'Gamma', #0.933, -> a_ref =  ps_prms["gamma_ref"] * ps_prms["cells_in_series"] * k_Boltzmann * 298.16 / q_electron,
# The temperature coefficient for the diode ideality factor, 1/K
# using 2nd value comparing in the diode ideality factor 
"mu_gamma" : 'muGamma', #'-0.0008' -0.001, 
#The light-generated current (or photocurrent) at reference conditions, in amperes.
# using Isc for I_L_ref
"I_L_ref" : 'Isc', #8.490, 
#The dark or diode reverse saturation current at reference conditions, in amperes.
"I_o_ref" : 0.040 / (10 ** 9), # NOT AVAILABLE IN THE FILE
#The shunt resistance at reference conditions, in ohms.
"R_sh_ref" : 'RShunt', #300, 
#The shunt resistance at zero irradiance conditions, in ohms.
"R_sh_0" : 'Rp_0', # 1200, 
#The series resistance at reference conditions, in ohms.
# series model
"R_s" : 'RSerie', #0.3, 
#The number of cells connected in series.
"cells_in_series" : 'NCelS', #60, 
#The exponent in the equation for shunt resistance, unitless. Defaults to 5.5.
# PVsyst: this value is set as default value ... Navigation:  Physical models used > PV Module - Standard one-diode-model >
"R_sh_exp" : 'Rp_Exp', #5.5, 
#The energy bandgap at reference temperature in units of eV. 1.121 eV for crystalline silicon. EgRef must be >0.
"EgRef" : 1.12, #1, 
# Reference irradiance in W/m^2.
"irrad_ref" : 'GRef', #1000, 
"temp_ref" : 25
}